[INDEX](Kubernetes.ipynb)  [NEXT](4.5.Traefik.ipynb)     [BELOW - LoadBalancer](#LoadBalancer)

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width=100 />

# 4. Kubernetes Services

# Exposing applications as a Service
<pre style="align:center" >
For this section of the lab we will continue to work with our N worker-node cluster.
</pre>
<img src="images/KubeNodes_1m_2w.svg.png" width="400" />
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>

## Services allow to abstract away Pod details
As mentioned earlier exposing our application as a Service allows to access the application at a single virtual ip address and port - without having to know the ip addresses of the individual pods which may change ip address as pods are scaled up/down or as pods or nodes fail or come on line.

It can also allow load-balancing across pods.

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @mjbright </td></tr></table>

In [2]:
kubectl get pods

NAME                        READY     STATUS    RESTARTS   AGE
k8s-demo-6b76bd84d7-qmzrl   1/1       Running   0          9m
k8s-demo-6b76bd84d7-vsnj9   1/1       Running   0          17m


In [7]:
kubectl delete deploy -l app=k8s-demo

deployment.extensions "k8s-demo" deleted


In [8]:
kubectl get pods

NAME                        READY     STATUS        RESTARTS   AGE
k8s-demo-6b76bd84d7-6nxrw   0/1       Terminating   0          43s
k8s-demo-6b76bd84d7-bqbzl   0/1       Terminating   0          44s


In [9]:
kubectl get pods

No resources found.


In [10]:
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080 

deployment.apps "k8s-demo" created


In [11]:
kubectl get pods

NAME                        READY     STATUS    RESTARTS   AGE
k8s-demo-6b76bd84d7-gn9vs   1/1       Running   0          16s
k8s-demo-6b76bd84d7-pk9mh   1/1       Running   0          16s


    
## Accessing our application - via a Service NodePort

We can expose our service using different types.

The first type we will consider is called NodePort and has the effect of exposing our Service on the same port on each Kubernetes node - and so with the ip address of that node.

Let's try that with our service:

In [12]:
kubectl expose deploy k8s-demo --type=NodePort --name=k8s-demo-service

service "k8s-demo-service" exposed


In [13]:
kubectl get svc/k8s-demo-service -o wide

EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ print $3;}')

echo; echo "----"
echo EXPOSED_PORT=$EXPOSED_PORT
echo CLUSTER_IP=$CLUSTER_IP

NAME               TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service   NodePort   10.0.125.184   <none>        8080:31909/TCP   18s       app=k8s-demo

----
EXPOSED_PORT=31909
CLUSTER_IP=10.0.125.184


If you are running inside a Pod on the AKS cluster you can already access the service using the CLUSTER-IP address and the internal port.

This is not using the NodePort.

In [ ]:
curl $CLUSTER_IP:8080

In AKS environment we do not know the Node ip addresses, if we did we could curl on

NODE_IP:EXPOSED_PORT

In [15]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment.extensions "k8s-demo" scaled


In [16]:
kubectl get all -o wide

NAME                            READY     STATUS    RESTARTS   AGE       IP           NODE
pod/k8s-demo-6b76bd84d7-d496w   1/1       Running   0          2m        10.244.8.8   aks-nodepool1-35637019-1
pod/k8s-demo-6b76bd84d7-gn9vs   1/1       Running   0          3m        10.244.4.8   aks-nodepool1-35637019-9
pod/k8s-demo-6b76bd84d7-pk9mh   1/1       Running   0          3m        10.244.6.8   aks-nodepool1-35637019-6

NAME                       TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE       SELECTOR
service/k8s-demo-service   NodePort    10.0.125.184   <none>        8080:31909/TCP   3m        app=k8s-demo
service/kubernetes         ClusterIP   10.0.0.1       <none>        443/TCP          16h       <none>

NAME                             DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deployment.extensions/k8s-demo   3         3         3            3           3m        k8s-demo     mjbright/k8s-demo:1   app=k

## NOTE: No pods are scheduled to the Master node

We would need to have a "tainted" Master node to allow Pods to be scheduled there (Minikube which is a single-node cluster obviously allows this).

Note below that the pods are deployed to Worker nodes kube-node-1 and kube-node-2.

In [17]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-d496w   1/1       Running   0          2m        10.244.8.8   aks-nodepool1-35637019-1
k8s-demo-6b76bd84d7-gn9vs   1/1       Running   0          3m        10.244.4.8   aks-nodepool1-35637019-9
k8s-demo-6b76bd84d7-pk9mh   1/1       Running   0          3m        10.244.6.8   aks-nodepool1-35637019-6


Let's try to access the Pods directly on the Worker nodes:

This will fail on AKS

In [21]:
NODE1_IP=$(kubectl describe node aks-nodepool1-35637019-1 | grep InternalIP: | sed 's/.*: *//')
NODE2_IP=$(kubectl describe node aks-nodepool1-35637019-2 | grep InternalIP: | sed 's/.*: *//')

echo "NODE1_IP=$NODE1_IP"
echo "NODE2_IP=$NODE2_IP"

NODE1_IP=10.240.0.8
NODE2_IP=10.240.0.11


Again this will fail on AKS as we don't have access to the Node IPs:

In [26]:
curl http://$NODE1_IP:$EXPOSED_PORT

In [27]:
curl http://$NODE2_IP:$EXPOSED_PORT

In [28]:
kubectl describe svc/k8s-demo-service

Name:                     k8s-demo-service
Namespace:                default
Labels:                   app=k8s-demo
Annotations:              <none>
Selector:                 app=k8s-demo
Type:                     NodePort
IP:                       10.0.125.184
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  31909/TCP
Endpoints:                10.244.4.8:8080,10.244.6.8:8080,10.244.8.8:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


#### For more information about using NodePort see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/service-access-application-cluster/)

**Note**: Although there is a Cluster IP there is no Service IP assigned, because we are using NodePort

So we can access our Pods by connecting to the hosting node but not to the Service ... so far ...

<a name="LoadBalancer"> </a>

## Accessing our application - via a Service LoadBalancer


In [29]:
#kubectl delete svc/k8s-demo-service
kubectl get svc | grep k8s-demo-service-lb && kubectl delete svc/k8s-demo-service-lb
#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb

#RS_K8SDEMO=$(kubectl get rs | grep k8s-demo | awk '{ print $1;}')
#kubectl expose rs $RS_K8SDEMO --type="LoadBalancer" --name="k8s-demo-service-lb"

K8SDEMO=$(kubectl get deploy | grep k8s-demo | awk '{ print $1;}')

#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb
kubectl expose deploy $K8SDEMO --type="LoadBalancer" --name="k8s-demo-service-lb"


service "k8s-demo-service-lb" exposed


### LoadBalancer not working due to lack of External-ips

We see below that on our cluster our EXTERNAL-IP remains un a pending state !!

In [30]:
kubectl get svc -o wide

NAME                  TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service      NodePort       10.0.125.184   <none>        8080:31909/TCP   9m        app=k8s-demo
k8s-demo-service-lb   LoadBalancer   10.0.44.174    <pending>     8080:32531/TCP   4s        app=k8s-demo
kubernetes            ClusterIP      10.0.0.1       <none>        443/TCP          16h       <none>


In [31]:
kubectl get svc -o wide

NAME                  TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service      NodePort       10.0.125.184   <none>        8080:31909/TCP   10m       app=k8s-demo
k8s-demo-service-lb   LoadBalancer   10.0.44.174    <pending>     8080:32531/TCP   1m        app=k8s-demo
kubernetes            ClusterIP      10.0.0.1       <none>        443/TCP          16h       <none>


In [ ]:
kubectl describe svc/k8s-demo-service-lb

LB_EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
LB_CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ print $3;}')

echo
echo LB_EXPOSED_PORT=$LB_EXPOSED_PORT
echo LB_CLUSTER_IP=$LB_CLUSTER_IP
echo MASTER_IP=$MASTER_IP

In [ ]:
curl ${LB_CLUSTER_IP}:$LB_EXPOSED_PORT

In [ ]:
curl ${MASTER_IP}:$EXPOSED_PORT

In [ ]:
kubectl describe no/kube-master | grep "InternalIP:"

MASTER_IP=$(kubectl describe no/kube-master | grep "InternalIP:" | sed -e 's/.*: *//')
echo MASTER_IP=$MASTER_IP
WORKER1_IP=$(kubectl describe no/kube-node-1 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER1_IP=$WORKER1_IP
WORKER2_IP=$(kubectl describe no/kube-node-2 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER2_IP=$WORKER2_IP

#curl $MASTER_IP:8080
curl $WORKER2_IP:8080
curl $WORKER2_IP:$EXPOSED_PORT
curl $WORKER1_IP:8080
curl $WORKER1_IP:$EXPOSED_PORT
curl $MASTER_IP:$EXPOSED_PORT

#### For more information about using LoadBalancer see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/load-balance-access-application-cluster/)


[INDEX](Kubernetes.ipynb)  [NEXT](4.5.Traefik.ipynb)
